In [5]:
from analysis.anomaly import prepare_data, fetch_anomalies_and_preceding_data

In [6]:
import pandas as pd
from json import load, dumps
import numpy as np

In [7]:
# # Anomaly Detection
#
# - Create a 'vitality' feature, which is an average of all the vitality metrics.
# - Plot a graph of this over time
# - Determine the standard deviation & mean
# - Identify specific points which fall out of (mean +/- sigma), and return the 7 day period immediately preceeding this (separate function).
#     - One function which detects anomalies, and another function which takes the anomalies, and uses the date feature to extract 7 (or otherwise specified) periods of measured metric data immediately preceding the anomaly's date.
#     - Quantify changes in correlated metrics (% change in the last few days compared to a global average or baseline)

import pandas as pd
import numpy as np
import math
# import matplotlib.pyplot as plt

# Data preparation
# data = pd.read_csv('sample_agg_health_and_checkin_data.csv')
#
# pd.read_json('sampleAggHealthAndCheckinData.json')

vitality_metrics = ["generalFeeling", "mood", "focus", "energy"]
measured_metrics = []

# Given a list of health data, engineers the 'vitality' feature, which is the average of all vitality metrics recorded
# for a given day, excluding '0' or NaN values for each row considered when applying a column-wise mean.
# The result is an array of shape (-1, 2), where each row contains the original data frame index, so that we can
# re-merge at a later stage and extract the measured_metrics, as well as the actual vitality for the day.
def summarise_vitality(data, vitality_metrics):
    vitality = np.array([ np.mean(x[np.logical_not(np.isnan(x))]) for x in data[vitality_metrics].values ]).reshape(-1, 1)
    indeces = np.array(data.index.to_series().values).reshape(-1, 1)
    return np.append(indeces, vitality, axis=1)


# Given a pd data frame containing aggregated health data, creates the 'vitality'
# feature, and interpolates weight to remove NaN values.
def prepare_data(data):
    # print(type(data))
    if type(data) != pd.core.frame.DataFrame:
        try:
            # print("Attempting to read from json.")
            data = pd.read_json(data)
        except:
            log("Failed to parse data as JSON.")
            try:
                # print("Attempting to read from csv.")
                data = pd.read_csv(data)
            except:
                log("Failed to parse data as CSV.")

                try:
                    data = pd.DataFrame.from_dict(data)
                except:
                    log("Failed to parse data as dict.")
                    pass

    # print(data)
    assert type(data) is pd.core.frame.DataFrame

    global measured_metrics

    # Remove 'attributesAndCounts' if present.
    if "attributesAndCounts" in data.columns:
        data = data.drop("attributesAndCounts", axis=1)

    # Reassign measured_metrics to be used across the package.
    measured_metrics = [col for col in data.columns.drop('startOfDate') if col not in vitality_metrics]

    data_df = data.copy()

    vitality = summarise_vitality(data, vitality_metrics)

    # Add vitality to the data.
    data_df["vitality"] = vitality[:, 1]

    # Replace all '0' values in weight by NaN values so that we can interpolate.
    # Fill in all missing NaN weight values by linearly interpolating.
    # data_df.loc[data_df["weight"] == 0, "weight"] = None
    data_df["weight"] = data_df["weight"].interpolate(limit_direction='both')
    data_df.loc[data_df["weight"] == 0, "weight"] = None
    data_df["weight"] = data_df["weight"].interpolate(limit_direction='both')
    
    # Drop all NaN rows.
    data_df = data_df.dropna()
    
    # Remove rows which have not been enriched with added health data (e.g. Apple Watch was not worn that day).
    data_df = data_df.loc[data_df["basalEnergyBurned"] != 0]

    return data_df.reset_index(drop=True)


# # Detecting anomalies.
# For each anomaly detected, return the immediate 'X' day period preceding the occurrance of the anomaly.
# We merge the vitality for each day with all the other metrics observed or measured on a particular day / row.
def get_data_preceding_anomaly(data, anomaly_idx, preceding_num_days=7):
    return data.iloc[(anomaly_idx-preceding_num_days if anomaly_idx > preceding_num_days else 0):anomaly_idx]

# For each measured metric preceding an anomaly, find the most correlated features which contribute to the anomaly itself.
# For this, we will need to append 'vitality' as an extra column on the pandas dataframe.
def get_most_correlated_complement(corr_matrix, metric):
    most_corr_metric, most_corr_val = ("", 0)
    for colName, value in corr_matrix[metric].iteritems():
        if (colName == metric or colName in vitality_metrics):
            continue
        if (abs(value) > abs(most_corr_val)):
            most_corr_metric, most_corr_val = (colName, value)
    return (most_corr_metric, most_corr_val)

# Given a correlation matrix and a desired metric, returns a list of metrics and their correlation
# scores, sorted in terms of the highest correlation.
def fetch_most_correlated_to(data_df, desired_metric):
#     print(data_df, desired_metric)
#     corr_matrix = data.corr()
    metrics_to_select = measured_metrics + [desired_metric]
#     print(f'Metrics to select: {metrics_to_select}')
#     print(f'Subset of measured {data_df[measured_metrics].columns}')
#     print(f'Subset of selected {data_df[metrics_to_select].columns}')
#     print(f'Correlation matrix for metrics_to_select {data_df[metrics_to_select].corr()}')
    correlation_matrix = data_df[metrics_to_select].corr()[desired_metric]
    correlation_matrix_df = pd.DataFrame(correlation_matrix)

#     print('Correlation Matrix', correlation_matrix_df)

    sorted_abs_corr_matrix_df = pd.DataFrame(correlation_matrix.abs()).sort_values(desired_metric, ascending=False)

#     print('Sorted (abs) corr matrix', sorted_abs_corr_matrix_df)

    # After sorting the correlation matrix using absolute values, we need to 'merge' back in the negative
    # values wherever they also apply. We keep all key in the sorted_abs_corr_matrix so that we can
    # preserve the index ordering which we computed during the sort method applied above.
    sorted_corr_matrix_df = sorted_abs_corr_matrix_df.join(other=correlation_matrix_df, on=sorted_abs_corr_matrix_df.index, how='left', lsuffix="_abs").drop(f'{desired_metric}_abs', axis=1)

#     print("Sorted corr matrix", sorted_corr_matrix_df)

    # Return all corrleated metrics, excluding 'vitality' which is trivially '1' by default.
    return sorted_corr_matrix_df

# Given a data frame, determines the % change in each column when compared to the global mean, as well as the
# start of the subset period, by fitting a line and determining the % change between the first and last point
# of the fitted line.
def determine_global_percentage_change_for_subset(all_data, subset_data):
    return pd.DataFrame((subset_data.mean() / all_data.mean()) - 1)

# Determines the local percentage change for a subset of the data, by fitting a line of best fit and determining
# the relative change between the first and the last value.
def determine_local_percentage_change_for_subset(subset_data):

    # if subset_data.empty:
    #     return subset_data

    x_mean = subset_data.index.values.mean()
    x_centered = (subset_data.index.values - x_mean)

    y_mean = subset_data.mean()
    y_centered = subset_data.drop("startOfDate", axis=1) - y_mean

    m = (x_centered.reshape(-1, 1) * y_centered).sum() / (x_centered ** 2).sum()

    b = y_mean - m * x_mean

    lobf_first_point = m * subset_data.index.values[0] + b
    lobf_last_point = m * subset_data.index.values[-1] + b

    local_percentage_change = lobf_last_point / lobf_first_point

    return pd.DataFrame(local_percentage_change - 1)

# Obtains correlated metrics along with the % change in each correlated metric from the global average.
# These are sorted by an 'importance' ranking which weights the correlation by the percentage change.
# Returns all metrics with an importance greater than or equal to 0.4.
def fetch_most_important_metrics_related_to(all_data, subset_data, desired_metric, importance_threshold=0.4):

    most_corr_metrics = fetch_most_correlated_to(subset_data, desired_metric)
    global_percentage_change_for_subset = determine_global_percentage_change_for_subset(all_data, subset_data)
    local_percentage_change_for_subset = determine_local_percentage_change_for_subset(subset_data)
    joined_df = most_corr_metrics
    joined_df["local_percentage_change"] = local_percentage_change_for_subset
    joined_df["global_percentage_change"] = global_percentage_change_for_subset
    joined_df.columns = ["correlation", "local_percentage_change", "global_percentage_change"]

    # Drop the desired_metric column.
    joined_df = joined_df.drop(desired_metric, axis=0)

    # Add an 'importance' column which weights each correlated metric by the percentage change which occured
    # (both globally, and locally).
    #
    # We take into account the percentage change, as a correlation in and of itself isn't as useful out side
    # of the context of a recent change. Indication of recent change would enable awareness and allow us to take action.
    joined_df['importance'] = joined_df["correlation"].abs() * joined_df["global_percentage_change"].abs() * joined_df["local_percentage_change"].abs()
    joined_df['importance'] = joined_df['importance'] / joined_df['importance'].max()

    return joined_df.sort_values(by='importance', ascending=False)

# Fetch the most correlated metrics, and display a comparison for each.
def fetch_most_correlated_pairs(data):
    corr = data.corr()
    correlated_metrics = []
    for col in corr.columns:
        pair, corrVal = get_most_correlated_complement(corr, col)
        if ((col, pair, corrVal) not in correlated_metrics or (pair, col, corrVal) not in correlated_metrics):
            correlated_metrics.append((col, pair, corrVal))
    return correlated_metrics

# Compare metrics which are most correlated to the desired metric visually, displaying the values for both
# metrics across time, as well as labelling the correlation between the two metrics.
# def compare_most_correlated_to_visually(data, desired_metric, max_num_correlated_metrics=3):

#     most_correlated_metrics = fetch_most_correlated_to(data, desired_metric)
#     most_correlated_metrics = list(zip(most_correlated_metrics.index, most_correlated_metrics.values.reshape(-1)))[1:]
#     if max_num_correlated_metrics > len(most_correlated_metrics):
#         max_num_correlated_metrics = len(most_correlated_metrics)

#     # Fetch the `max_num_correlated_metrics` top most correlated metrics, skipping the first which is
#     # trivially the desired_metric itself.
#     most_correlated_metrics = most_correlated_metrics[0:max_num_correlated_metrics]

#     plot_idx = 0
#     for correlated_metric, corr_score in most_correlated_metrics:
#         plt.figure(plot_idx)
#         compare_metrics(data, desired_metric, correlated_metric, corr_score=corr_score)
#         plot_idx += 1

# def compare_most_correlated_visually(data):
#     correlated_metrics = fetch_most_correlated_pairs(data)
#     plotIndex = 0
#     for (metricOne, metricTwo, _) in correlated_metrics:
#         plt.figure(plotIndex)
#         plotIndex += 1
#         compare_metrics(data, metricOne, metricTwo)

# Plotting restingHeartRate against the 4 vitality indicators.
# def compare_metrics(df, metricOne, metricTwo, corr_score=None):
#     feelingVHR = df[[metricOne, metricTwo]].dropna()
#     metricOneMax = feelingVHR[metricOne].max()
#     metricTwoMax = feelingVHR[metricTwo].max()
#     plt.plot(feelingVHR[metricOne] / metricOneMax)
#     plt.plot(feelingVHR[metricTwo] / metricTwoMax)
#     plt.xlabel('Day')
#     plt.ylabel('Normalised value')
#     if corr_score is not None:
#         plt.text((feelingVHR.index.min() + feelingVHR.index.max()) / 2, 1, f'Correlation: {corr_score}', horizontalalignment='center', verticalalignment='center')
#     plt.legend()

# Given the data, fetch the anomalies and the preceding data as an array of dictionaries
# {
#     anomaly_idx,
#     anomaly_value,
#     preceding_data,
#     most_important_metrics,
#     most_important_preceding_data
# }
def fetch_anomalies_and_preceding_data(data, desired_metric="vitality", preceding_num_days=7, std_deviations=1, importance_threshold=0.4):

    if desired_metric is None:
        desired_metric = "vitality"

    if preceding_num_days is None:
        preceding_num_days = 7

    if std_deviations is None:
        std_deviations = 1

    if importance_threshold is None:
        importance_threshold = 0.4

    data = prepare_data(data)

    anomalies = []

    anomaly_idxs = detect_anomaly_indeces(data=data, desired_metric=desired_metric, std_deviations=std_deviations)

    anomaly_values = [data[desired_metric].iloc[idx] for idx in anomaly_idxs]

    for anomaly_index, anomaly_value in zip(anomaly_idxs, anomaly_values):
        preceding_data = get_data_preceding_anomaly(data, anomaly_index, preceding_num_days=preceding_num_days)

        # If the preceding data is empty for whatever reason, skip the current anomaly.
        if preceding_data.empty:
            log(f'Preceding data empty for anomaly index {anomaly_index} and value {anomaly_value}.')
            continue

        important_metrics = fetch_most_important_metrics_related_to(data, preceding_data, desired_metric)
        most_important_metrics = important_metrics.loc[important_metrics["importance"] >= 0.4]
        most_important_preceding_data = preceding_data[most_important_metrics.index.values]

        anomalies.append({
            "desired_metric": desired_metric,
            "anomaly_index": int(anomaly_index),
            "anomaly_value": float(anomaly_value),
            "preceding_data": preceding_data.to_dict(),
            "most_important_metrics": most_important_metrics.to_dict(),
            "most_important_preceding_data": most_important_preceding_data.to_dict()
        })

    return anomalies

def log(message):
    print(f'ANOMLAY | {message}')


In [10]:
data = pd.read_json("../data/sample_data.json")
# with open('./sample_data.json', 'r+') as json_file:
#     json_dict = load(json_file)
# #     print(json_dict["result"])
#     json_file.seek(0)
#     json_file.truncate(0)
#     json_file.write(dumps(json_dict["result"]))

In [11]:
data_prepped = prepare_data(data)

In [12]:
data_prepped.loc[data_prepped["caloricIntake"] == 0]

,startOfDate,generalFeeling,mood,energy,focus,activeEnergyBurned,basalEnergyBurned,caloricIntake,dietaryCarbohydrates,dietaryFats,dietaryProtein,hrv,lowHeartRateEvents,restingHeartRate,sleepHours,weight,vitality
26,2020-05-18T00:00:00.000Z,3.000000,4.000000,3.000000,3.000000,179.670,1303.657,0.0,0.0,0.0,0.0,77.330400,1.0,40.0,8.183194,59.746013,3.250000
38,2020-04-09T00:00:00.000Z,2.000000,2.000000,2.000000,2.000000,246.898,1447.999,0.0,0.0,0.0,0.0,66.439715,1.0,40.0,7.252083,59.800052,2.000000
39,2020-04-08T00:00:00.000Z,2.000000,2.000000,2.000000,2.000000,138.352,1610.592,0.0,0.0,0.0,0.0,70.087394,11.0,36.0,10.357500,59.600121,2.000000
40,2020-04-07T00:00:00.000Z,2.500000,3.000000,2.500000,2.500000,203.119,1610.662,0.0,0.0,0.0,0.0,63.690593,5.0,35.0,8.752500,59.400190,2.625000
42,2020-04-05T00:00:00.000Z,3.000000,3.000000,2.500000,3.000000,271.611,1491.387,0.0,0.0,0.0,0.0,49.691969,3.0,35.0,7.500000,59.600221,2.875000
44,2020-03-24T00:00:00.000Z,3.000000,3.000000,3.000000,2.000000,221.724,1631.619,0.0,0.0,0.0,0.0,62.045726,3.0,40.0,6.158056,59.800105,2.750000
51,2020-03-12T00:00:00.000Z,1.000000,2.000000,1.000000,2.000000,214.771,1645.249,0.0,0.0,0.0,0.0,66.670102,2.0,40.0,8.316944,60.149902,1.500000
54,2020-03-05T00:00:00.000Z,4.000000,4.000000,3.666667,3.500000,15.157,442.473,0.0,0.0,0.0,0.0,72.246145,3.0,39.0,0.000000,60.866659,3.791667
56,2020-03-03T00:00:00.000Z,5.000000,5.000000,4.500000,4.000000,213.533,586.183,0.0,0.0,0.0,0.0,66.624398,0.0,41.0,7.478056,61.000016,4.625000
57,2020-03-01T00:00:00.000Z,2.500000,3.000000,2.500000,3.000000,27.204,643.883,0.0,0.0,0.0,0.0,54.056139,0.0,41.0,0.000000,60.799980,2.750000


In [13]:
# # Anomaly Detection
#
# - Create a 'vitality' feature, which is an average of all the vitality metrics.
# - Plot a graph of this over time
# - Determine the standard deviation & mean
# - Identify specific points which fall out of (mean +/- sigma), and return the 7 day period immediately preceeding this (separate function).
#     - One function which detects anomalies, and another function which takes the anomalies, and uses the date feature to extract 7 (or otherwise specified) periods of measured metric data immediately preceding the anomaly's date.
#     - Quantify changes in correlated metrics (% change in the last few days compared to a global average or baseline)

import pandas as pd
import numpy as np
import math
# import matplotlib.pyplot as plt

# Data preparation
# data = pd.read_csv('sample_agg_health_and_checkin_data.csv')
#
# pd.read_json('sampleAggHealthAndCheckinData.json')

vitality_metrics = ["generalFeeling", "mood", "focus", "energy"]
measured_metrics = []

# Given a list of health data, engineers the 'vitality' feature, which is the average of all vitality metrics recorded
# for a given day, excluding '0' or NaN values for each row considered when applying a column-wise mean.
# The result is an array of shape (-1, 2), where each row contains the original data frame index, so that we can
# re-merge at a later stage and extract the measured_metrics, as well as the actual vitality for the day.
def summarise_vitality(data, vitality_metrics):
    vitality = np.array([ np.mean(x[np.logical_not(np.isnan(x))]) for x in data[vitality_metrics].values ]).reshape(-1, 1)
    indeces = np.array(data.index.to_series().values).reshape(-1, 1)
    return np.append(indeces, vitality, axis=1)


# Given a pd data frame containing aggregated health data, creates the 'vitality'
# feature, and interpolates weight to remove NaN values.
def prepare_data(data):
    # print(type(data))
    if type(data) != pd.core.frame.DataFrame:
        try:
            # print("Attempting to read from json.")
            data = pd.read_json(data)
        except:
            log("Failed to parse data as JSON.")
            try:
                # print("Attempting to read from csv.")
                data = pd.read_csv(data)
            except:
                log("Failed to parse data as CSV.")

                try:
                    data = pd.DataFrame.from_dict(data)
                except:
                    log("Failed to parse data as dict.")
                    pass

    # print(data)
    assert type(data) is pd.core.frame.DataFrame

    global measured_metrics

    # Remove 'attributesAndCounts' if present.
    if "attributesAndCounts" in data.columns:
        data = data.drop("attributesAndCounts", axis=1)

    # Reassign measured_metrics to be used across the package.
    measured_metrics = [col for col in data.columns.drop('startOfDate') if col not in vitality_metrics]

    data_df = data.copy()

    vitality = summarise_vitality(data, vitality_metrics)

    # Add vitality to the data.
    data_df["vitality"] = vitality[:, 1]

    # Replace all '0' values in weight by NaN values so that we can interpolate.
    # Fill in all missing NaN weight values by linearly interpolating.
    # data_df.loc[data_df["weight"] == 0, "weight"] = None
    data_df["weight"] = data_df["weight"].interpolate(limit_direction='both')
    data_df.loc[data_df["weight"] == 0, "weight"] = None
    data_df["weight"] = data_df["weight"].interpolate(limit_direction='both')
    
    # Drop all NaN rows.
    data_df = data_df.dropna()
    
    # Remove rows which have not been enriched with added health data (e.g. Apple Watch was not worn that day).
    data_df = data_df.loc[data_df["basalEnergyBurned"] != 0]

    return data_df.reset_index(drop=True)


# # Detecting anomalies.

# Detects anomalies in the data by detecting values which fall outside of the
# mu +/- sigma range, returning the indeces of the corresponding data frame
# rows where the desired metric falls outside of this range.
def detect_anomaly_indeces(data, desired_metric, std_deviations=1):

    sigma = data[desired_metric].std()
    mu = data[desired_metric].mean()

    upper_bound = mu + sigma*std_deviations
    lower_bound = mu - sigma*std_deviations

    anomaly_indeces = np.where((data[desired_metric] != 0) & (data[desired_metric] < lower_bound) | (data[desired_metric] > upper_bound))
    return anomaly_indeces[0]

# For each measured metric preceding an anomaly, find the most correlated features which contribute to the anomaly itself.
# For this, we will need to append 'vitality' as an extra column on the pandas dataframe.
def get_most_correlated_complement(corr_matrix, metric):
    most_corr_metric, most_corr_val = ("", 0)
    for colName, value in corr_matrix[metric].iteritems():
        if (colName == metric or colName in vitality_metrics):
            continue
        if (abs(value) > abs(most_corr_val)):
            most_corr_metric, most_corr_val = (colName, value)
    return (most_corr_metric, most_corr_val)

# Given a correlation matrix and a desired metric, returns a list of metrics and their correlation
# scores, sorted in terms of the highest correlation.
def fetch_most_correlated_to(data_df, desired_metric):
#     print(data_df, desired_metric)
#     corr_matrix = data.corr()
    metrics_to_select = measured_metrics + [desired_metric]
#     print(f'Metrics to select: {metrics_to_select}')
#     print(f'Subset of measured {data_df[measured_metrics].columns}')
#     print(f'Subset of selected {data_df[metrics_to_select].columns}')
#     print(f'Correlation matrix for metrics_to_select {data_df[metrics_to_select].corr()}')
    correlation_matrix = data_df[metrics_to_select].corr()[desired_metric]
    correlation_matrix_df = pd.DataFrame(correlation_matrix)

#     print('Correlation Matrix', correlation_matrix_df)

    sorted_abs_corr_matrix_df = pd.DataFrame(correlation_matrix.abs()).sort_values(desired_metric, ascending=False)

#     print('Sorted (abs) corr matrix', sorted_abs_corr_matrix_df)

    # After sorting the correlation matrix using absolute values, we need to 'merge' back in the negative
    # values wherever they also apply. We keep all key in the sorted_abs_corr_matrix so that we can
    # preserve the index ordering which we computed during the sort method applied above.
    sorted_corr_matrix_df = sorted_abs_corr_matrix_df.join(other=correlation_matrix_df, on=sorted_abs_corr_matrix_df.index, how='left', lsuffix="_abs").drop(f'{desired_metric}_abs', axis=1)

#     print("Sorted corr matrix", sorted_corr_matrix_df)

    # Return all corrleated metrics, excluding 'vitality' which is trivially '1' by default.
    return sorted_corr_matrix_df

# Given a data frame, determines the % change in each column when compared to the global mean, as well as the
# start of the subset period, by fitting a line and determining the % change between the first and last point
# of the fitted line.
def determine_global_percentage_change_for_subset(all_data, subset_data):
    return pd.DataFrame((subset_data.mean() / all_data.mean()) - 1)

# Determines the local percentage change for a subset of the data, by fitting a line of best fit and determining
# the relative change between the first and the last value.
def determine_local_percentage_change_for_subset(subset_data):

    # if subset_data.empty:
    #     return subset_data

    x_mean = subset_data.index.values.mean()
    x_centered = (subset_data.index.values - x_mean)

    y_mean = subset_data.mean()
    y_centered = subset_data.drop("startOfDate", axis=1) - y_mean

    m = (x_centered.reshape(-1, 1) * y_centered).sum() / (x_centered ** 2).sum()

    b = y_mean - m * x_mean

    lobf_first_point = m * subset_data.index.values[0] + b
    lobf_last_point = m * subset_data.index.values[-1] + b

    local_percentage_change = lobf_last_point / lobf_first_point

    return pd.DataFrame(local_percentage_change - 1)

# Obtains correlated metrics along with the % change in each correlated metric from the global average.
# These are sorted by an 'importance' ranking which weights the correlation by the percentage change.
# Returns all metrics with an importance greater than or equal to 0.4.
def fetch_most_important_metrics_related_to(all_data, subset_data, desired_metric, importance_threshold=0.4):

    most_corr_metrics = fetch_most_correlated_to(subset_data, desired_metric)
    global_percentage_change_for_subset = determine_global_percentage_change_for_subset(all_data, subset_data)
    local_percentage_change_for_subset = determine_local_percentage_change_for_subset(subset_data)
    joined_df = most_corr_metrics
    joined_df["local_percentage_change"] = local_percentage_change_for_subset
    joined_df["global_percentage_change"] = global_percentage_change_for_subset
    joined_df.columns = ["correlation", "local_percentage_change", "global_percentage_change"]

    # Drop the desired_metric column.
    joined_df = joined_df.drop(desired_metric, axis=0)

    # Add an 'importance' column which weights each correlated metric by the percentage change which occured
    # (both globally, and locally).
    #
    # We take into account the percentage change, as a correlation in and of itself isn't as useful out side
    # of the context of a recent change. Indication of recent change would enable awareness and allow us to take action.
    joined_df['importance'] = joined_df["correlation"].abs() * joined_df["global_percentage_change"].abs() * joined_df["local_percentage_change"].abs()
    joined_df['importance'] = joined_df['importance'] / joined_df['importance'].max()

    return joined_df.sort_values(by='importance', ascending=False)

# Fetch the most correlated metrics, and display a comparison for each.
def fetch_most_correlated_pairs(data):
    corr = data.corr()
    correlated_metrics = []
    for col in corr.columns:
        pair, corrVal = get_most_correlated_complement(corr, col)
        if ((col, pair, corrVal) not in correlated_metrics or (pair, col, corrVal) not in correlated_metrics):
            correlated_metrics.append((col, pair, corrVal))
    return correlated_metrics

# Compare metrics which are most correlated to the desired metric visually, displaying the values for both
# metrics across time, as well as labelling the correlation between the two metrics.
# def compare_most_correlated_to_visually(data, desired_metric, max_num_correlated_metrics=3):

#     most_correlated_metrics = fetch_most_correlated_to(data, desired_metric)
#     most_correlated_metrics = list(zip(most_correlated_metrics.index, most_correlated_metrics.values.reshape(-1)))[1:]
#     if max_num_correlated_metrics > len(most_correlated_metrics):
#         max_num_correlated_metrics = len(most_correlated_metrics)

#     # Fetch the `max_num_correlated_metrics` top most correlated metrics, skipping the first which is
#     # trivially the desired_metric itself.
#     most_correlated_metrics = most_correlated_metrics[0:max_num_correlated_metrics]

#     plot_idx = 0
#     for correlated_metric, corr_score in most_correlated_metrics:
#         plt.figure(plot_idx)
#         compare_metrics(data, desired_metric, correlated_metric, corr_score=corr_score)
#         plot_idx += 1

# def compare_most_correlated_visually(data):
#     correlated_metrics = fetch_most_correlated_pairs(data)
#     plotIndex = 0
#     for (metricOne, metricTwo, _) in correlated_metrics:
#         plt.figure(plotIndex)
#         plotIndex += 1
#         compare_metrics(data, metricOne, metricTwo)

# Plotting restingHeartRate against the 4 vitality indicators.
# def compare_metrics(df, metricOne, metricTwo, corr_score=None):
#     feelingVHR = df[[metricOne, metricTwo]].dropna()
#     metricOneMax = feelingVHR[metricOne].max()
#     metricTwoMax = feelingVHR[metricTwo].max()
#     plt.plot(feelingVHR[metricOne] / metricOneMax)
#     plt.plot(feelingVHR[metricTwo] / metricTwoMax)
#     plt.xlabel('Day')
#     plt.ylabel('Normalised value')
#     if corr_score is not None:
#         plt.text((feelingVHR.index.min() + feelingVHR.index.max()) / 2, 1, f'Correlation: {corr_score}', horizontalalignment='center', verticalalignment='center')
#     plt.legend()

# Given the data, fetch the anomalies and the preceding data as an array of dictionaries
# {
#     anomaly_idx,
#     anomaly_value,
#     preceding_data,
#     most_important_metrics,
#     most_important_preceding_data
# }
def fetch_anomalies_and_preceding_data(data, desired_metric="vitality", preceding_num_days=7, std_deviations=1, importance_threshold=0.4):

    if desired_metric is None:
        desired_metric = "vitality"

    if preceding_num_days is None:
        preceding_num_days = 7

    if std_deviations is None:
        std_deviations = 1

    if importance_threshold is None:
        importance_threshold = 0.4

    data = prepare_data(data)

    anomalies = []

    anomaly_idxs = detect_anomaly_indeces(data=data, desired_metric=desired_metric, std_deviations=std_deviations)

    anomaly_values = [data[desired_metric].iloc[idx] for idx in anomaly_idxs]

    for anomaly_index, anomaly_value in zip(anomaly_idxs, anomaly_values):
        preceding_data = get_data_preceding_anomaly(data, anomaly_index, preceding_num_days=preceding_num_days)

        # If the preceding data is empty for whatever reason, skip the current anomaly.
        if preceding_data.empty:
            log(f'Preceding data empty for anomaly index {anomaly_index} and value {anomaly_value}.')
            continue

        important_metrics = fetch_most_important_metrics_related_to(data, preceding_data, desired_metric)
        most_important_metrics = important_metrics.loc[important_metrics["importance"] >= 0.4]
        most_important_preceding_data = preceding_data[most_important_metrics.index.values]
        
        # If we do not have enough preceding data to cover the preceding number of days, then we should 
        # skip this particular result.
#         if preceding_data

        anomalies.append({
            "desired_metric": desired_metric,
            "anomaly_index": int(anomaly_index),
            "anomaly_value": float(anomaly_value),
            "preceding_data": preceding_data.to_dict(),
            "most_important_metrics": most_important_metrics.to_dict(),
            "most_important_preceding_data": most_important_preceding_data.to_dict()
        })

    return anomalies

def log(message):
    print(f'ANOMLAY | {message}')

In [14]:
detect_anomaly_indeces(data, "sleepHours")

array([  8,  52,  65,  67,  68,  89,  90, 104, 112, 124, 126, 129, 140])

In [15]:
anomalies = fetch_anomalies_and_preceding_data(data)

ANOMLAY | Preceding data empty for anomaly index 0 and value 1.75.


In [16]:
np.where(((data["caloricIntake"] != 0) & (data["caloricIntake"] > 3200)))

(array([ 74, 108, 136]),)

In [22]:
data.iloc[9]["startOfDate"]

'2020-06-27T00:00:00.000Z'

In [17]:
# For each anomaly detected, return the immediate 'X' day period preceding the occurrance of the anomaly.
# We merge the vitality for each day with all the other metrics observed or measured on a particular day / row.
def get_data_preceding_anomaly(data, anomaly_idx, preceding_num_days=7):
    return data.iloc[anomaly_idx: min(anomaly_idx+preceding_num_days, len(data))]
#     return data.iloc[(anomaly_idx-preceding_num_days if anomaly_idx > preceding_num_days else 0):anomaly_idx]


In [18]:
preceding_data = get_data_preceding_anomaly(data, 10, preceding_num_days=7)

In [19]:
preceding_data.head(3)

,startOfDate,generalFeeling,mood,energy,focus,attributesAndCounts,activeEnergyBurned,basalEnergyBurned,caloricIntake,dietaryCarbohydrates,dietaryFats,dietaryProtein,hrv,lowHeartRateEvents,restingHeartRate,sleepHours,weight
10,2020-06-26T00:00:00.000Z,2.0,2.0,2.0,2.0,"{'generalFeeling': 1, 'mood': 1, 'energy': 1, ...",749.137,1598.732,1760.089325,112.762424,65.292268,157.223942,60.377090,3.0,39.0,9.622778,0.000000
11,2020-06-25T00:00:00.000Z,1.0,1.0,1.0,1.0,"{'generalFeeling': 1, 'mood': 1, 'energy': 1, ...",551.658,1606.791,2174.395233,197.696106,70.343836,182.788260,69.829800,1.0,40.0,8.825694,0.000000
12,2020-06-24T00:00:00.000Z,2.0,3.0,2.0,2.0,"{'generalFeeling': 1, 'mood': 1, 'energy': 1, ...",265.060,1582.153,1888.463425,143.513682,80.526786,143.098830,56.054286,18.0,37.0,9.568056,57.200267


In [32]:
preceding_data.where(preceding_data != 0, None).dropna()

,startOfDate,generalFeeling,mood,energy,focus,attributesAndCounts,activeEnergyBurned,basalEnergyBurned,caloricIntake,dietaryCarbohydrates,dietaryFats,dietaryProtein,hrv,lowHeartRateEvents,restingHeartRate,sleepHours,weight
12,2020-06-24T00:00:00.000Z,2,3,2,2,"{'generalFeeling': 1, 'mood': 1, 'energy': 1, ...",265.06,1582.15,1888.46,143.514,80.5268,143.099,56.0543,18,37,9.56806,57.2003
13,2020-06-23T00:00:00.000Z,1,1,1,1,"{'generalFeeling': 1, 'mood': 1, 'energy': 1, ...",317.569,1582.59,1685.14,122.288,80.5868,108.147,54.1322,1,39,7.05458,57.8


In [66]:
# Given a dataframe, creates a dict for each column as arrays rather than objects keyed by the index.
def pandas_to_array_dict(df):
    
    output_dict = df.to_dict()
    
    for key in output_dict:
        output_dict[key] = list(output_dict[key].values())
    
    return output_dict

Inverting the `most_imporant_metrics` dictionary.

In [60]:
mims = anomalies[5]["most_important_metrics"]

# We want the keys of each value to be the new keys, and the current keys to exist within the key of each new key, with the old values place there. Effectively we want to interchange the keys for the two levels.
# dict((value, key) for value, key in zip(mims.values(), mims.keys()))

top_level_keys = list(mims.values())[0].keys()
inner_keys = list(mims.keys())

output = {}

# lowHeartRateEvents, weight, etc
for top_level_key in list(mims.values())[0].keys():
    output[top_level_key] = {}
    # correlation, local_percentage_change, importance, etc
    for inner_key in mims.keys():
        output[top_level_key][inner_key] = mims[inner_key][top_level_key]

label = 'metric'
array = sorted(({**value, label:key } for key, value in zip(output.keys(), output.values())), key=lambda item: item["importance"], reverse=True)

[{'correlation': 0.08448214307493142,
  'local_percentage_change': 2.1000000000000005,
  'global_percentage_change': 0.9467382328654004,
  'importance': 1.0,
  'metric': 'lowHeartRateEvents'},
 {'correlation': 0.3560415680299623,
  'local_percentage_change': 0.5244936100097013,
  'global_percentage_change': 0.5069367453780269,
  'importance': 0.5636124084662236,
  'metric': 'dietaryFats'}]

In [169]:
# Inverts the inner and outer keys of a two-dimensional dictionary.
def invert_dictionary(dictionary):
    output = {}
    for top_level_key in list(dictionary.values())[0].keys():
        output[top_level_key] = {}
        for inner_key in dictionary.keys():
            output[top_level_key][inner_key] = dictionary[inner_key][top_level_key]
    return output

# Converts a dictionary into an array of values, sorted by a given key.
def dict_to_array(dictionary, label, sorted_by_key):
    return sorted(({**value, label:key } for key, value in zip(dictionary.keys(), dictionary.values())), key=lambda item: item[sorted_by_key], reverse=True)

def fetch_most_important_metrics_related_to(all_data, subset_data, desired_metric, importance_threshold=0.4):

    most_corr_metrics = fetch_most_correlated_to(subset_data, desired_metric)
    global_percentage_change_for_subset = determine_global_percentage_change_for_subset(all_data, subset_data)
    local_percentage_change_for_subset = determine_local_percentage_change_for_subset(subset_data)
    joined_df = most_corr_metrics
    joined_df["local_percentage_change"] = local_percentage_change_for_subset
    joined_df["global_percentage_change"] = global_percentage_change_for_subset
    joined_df.columns = ["correlation", "local_percentage_change", "global_percentage_change"]

    # Drop the desired_metric column.
    # joined_df = joined_df.drop(desired_metric, axis=0)

    # Add an 'importance' column which weights each correlated metric by the percentage change which occured
    # (both globally, and locally).
    #
    # We take into account the percentage change, as a correlation in and of itself isn't as useful out side
    # of the context of a recent change. Indication of recent change would enable awareness and allow us to take action.
    joined_df['importance'] = joined_df["correlation"].abs() * joined_df["global_percentage_change"].abs() * joined_df["local_percentage_change"].abs()
    joined_df['importance'] = joined_df['importance'] / joined_df['importance'].max()
    sorted_df = joined_df.sort_values(by='importance', ascending=False)

    # Separate the desired metric from the rest.
    important_metrics, anomaly_metrics = sorted_df.drop(desired_metric, axis=0), sorted_df.loc[desired_metric]

    return important_metrics, anomaly_metrics

def fetch_anomalies_and_preceding_data(data, desired_metric="vitality", preceding_num_days=7, std_deviations=1, importance_threshold=0.4):

    if desired_metric is None:
        desired_metric = "vitality"

    if preceding_num_days is None:
        preceding_num_days = 7

    if std_deviations is None:
        std_deviations = 1

    if importance_threshold is None:
        importance_threshold = 0.4

    data = prepare_data(data)

    anomalies = []

    anomaly_idxs = detect_anomaly_indeces(data=data, desired_metric=desired_metric, std_deviations=std_deviations)

    anomaly_values = [data[desired_metric].iloc[idx] for idx in anomaly_idxs]

    for anomaly_index, anomaly_value in zip(anomaly_idxs, anomaly_values):

        # The first element in the preceding data corresponds to the anomaly index. We add the 'startOfDate' as a top-level 
        # field to the returned dictionary.
        preceding_data = get_data_preceding_anomaly(data, anomaly_index, preceding_num_days=preceding_num_days)

        anomaly_row = preceding_data.iloc[0]
        anomaly_start_of_date = anomaly_row["startOfDate"]

        # If the preceding data is empty for whatever reason, skip the current anomaly.
        if preceding_data.empty:
            log(f'Preceding data empty for anomaly index {anomaly_index} and value {anomaly_value}.')
            continue
            
        # If we do not have enough preceding data to cover the preceding number of days, then we should 
        # skip this particular result.
        if len(preceding_data) < preceding_num_days:
            continue
            
        # Reverse and reindex the data.
        preceding_data = preceding_data.reindex(index=preceding_data.index[::-1])
        preceding_data_dict = pandas_to_array_dict(preceding_data)

        important_metrics, anomaly_metrics = fetch_most_important_metrics_related_to(data, preceding_data, desired_metric)
        most_important_metrics = important_metrics.loc[important_metrics["importance"] >= 0.4]
        most_important_metrics_dict = invert_dictionary(most_important_metrics.to_dict())
        most_important_metrics_array = dict_to_array(dictionary=most_important_metrics_dict, label="metric", sorted_by_key="importance")

        most_important_preceding_data = preceding_data[most_important_metrics.index.values]
        most_important_preceding_data_dict = pandas_to_array_dict(most_important_preceding_data)

        anomalies.append({
            "desired_metric": desired_metric,
            "anomaly_index": int(anomaly_index),
            "anomaly_value": float(anomaly_value),
            "anomaly_metrics": anomaly_metrics.to_dict(),
            "anomaly_start_of_date": anomaly_start_of_date,
            "preceding_data": preceding_data_dict,
            "most_important_metrics_dict": most_important_metrics_dict,
            "most_important_metrics_array": most_important_metrics_array,
            "most_important_preceding_data": most_important_preceding_data_dict
        })

    # Sort the anomalies by date.
    anomalies_sorted = sorted(anomalies, key=lambda anomaly: dateutil.parser.isoparse(anomaly["anomaly_start_of_date"]), reverse=True)

    return anomalies_sorted

In [170]:
anomalies = fetch_anomalies_and_preceding_data(data)

In [177]:
anomalies[5]["anomaly_start_of_date"]

'2020-06-25T00:00:00.000Z'

In [167]:
anomalies[0]["anomaly_metrics"]

{'correlation': 1.0,
 'local_percentage_change': 1.0211046250561298,
 'global_percentage_change': 0.04119081410933867,
 'importance': 0.12895636978312774}

In [ ]:
anomalies.sort()

In [162]:
import dateutil
start_of_date = anomalies[0]["anomaly_start_of_date"]
parsed = dateutil.parser.isoparse(start_of_date)
parsed

datetime.datetime(2020, 7, 4, 0, 0, tzinfo=tzutc())

Detecting anomaly indeces by a given date.

In [147]:
from datetime import datetime
def detect_anomaly_indeces(data, desired_metric, std_deviations=1, date=None):

    sigma = data[desired_metric].std()
    mu = data[desired_metric].mean()

    upper_bound = mu + sigma*std_deviations
    lower_bound = mu - sigma*std_deviations

    if date is None:
        anomaly_indeces = np.where((data[desired_metric] != 0) & (data[desired_metric] < lower_bound) | (data[desired_metric] > upper_bound))
    else: 
        anomaly_indeces = np.where(
            (data[desired_metric] != 0) & 
            ((data[desired_metric] < lower_bound) | (data[desired_metric] > upper_bound)) &
            (data["startOfDate"] == date)
        )

    return anomaly_indeces[0]

In [153]:
detect_anomaly_indeces(data, "mood", date="2020-07-01T00:00:00.000Z")

array([5])

In [152]:
data.iloc[5]

startOfDate                                      2020-07-01T00:00:00.000Z
generalFeeling                                                          2
mood                                                                    2
energy                                                                  1
focus                                                                   1
attributesAndCounts     {'generalFeeling': 1, 'mood': 1, 'energy': 1, ...
activeEnergyBurned                                                485.684
basalEnergyBurned                                                 1576.85
caloricIntake                                                     1413.21
dietaryCarbohydrates                                              108.996
dietaryFats                                                       54.6279
dietaryProtein                                                    121.065
hrv                                                               76.0992
lowHeartRateEvents                    